In [1]:
import local_load_data as ldata
import local_param as pm
import clean_data as cdata
import warnings
warnings.filterwarnings("ignore")
data_All_years = ldata.start_load(pm.csv_file)


Total Population involed in all traffic accidents:  6772563

Total number of all traffic accidents:  2570235


In [2]:
#data_years = ldata.get_TrafficData_CSV_year(data_All_years,pm.y_start,pm.y_end)

In [3]:
data_cleaned = cdata.data_clean_columns(data_All_years,pm.columns_clean,v_num=pm.v_num)
data_cleaned = data_cleaned[data_cleaned['P_ISEV']!='3']
data_cleaned['P_ISEV'].value_counts()
data_cleaned = cdata.modify_data(data_cleaned,pm.columns_grp)
cdata.get_data_stats(data_cleaned,pm.columns_stats)

C_YEAR column data is distributed as below: 

2015    0.080390
2016    0.078287
2010    0.068627
2011    0.067871
2012    0.067041
2004    0.064884
2017    0.060462
2014    0.059503
2008    0.057412
2013    0.052013
2003    0.051197
2009    0.049473
2007    0.048054
2001    0.042072
2002    0.040886
2006    0.040580
2005    0.040144
2000    0.015717
1999    0.015386
Name: C_YEAR, dtype: float64
Total size : (1081854,)
C_VEHS column data is distributed as below: 

2    1.0
Name: C_VEHS, dtype: float64
Total size : (1081854,)
C_RSUR column data is distributed as below: 

1    0.709982
2    0.205575
3    0.036334
5    0.034770
4    0.011893
6    0.000988
7    0.000327
8    0.000118
9    0.000013
Name: C_RSUR, dtype: float64
Total size : (1081854,)
C_RALN column data is distributed as below: 

1    0.817582
2    0.102672
3    0.044204
4    0.025169
5    0.006355
6    0.004019
Name: C_RALN, dtype: float64
Total size : (1081854,)
C_WTHR column data is distributed as below: 

1    0.734039
3 

In [4]:
import creat_ML_data as MLdata
data_selected = MLdata.create_data_for_ML(data_cleaned, pm.traffic_data_headers, pm.dummy_fields, pm.regression)
test_data, train_x, test_x, train_y, test_y = MLdata.train_test_data(data_selected,pm.time,pm.test_size,pm.year,pm.interval,pm.traffic_data_headers)
from sklearn.utils import class_weight
import numpy as np
class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)

array([1.06588515, 0.94178576])

In [5]:
import LR_model_present_2y as LRmp
import Tree_model_present as Tmp
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
if pm.regression =='lr':
  file_name = 't_{}_tst{}_trn{}_Csize_{}_pnt_{}_cw_{}_mltc_{}_svr_{}_cv_{}'.format(pm.time,pm.year,pm.interval,len(pm.C),pm.penalty,pm.classweight,pm.multi_class,pm.solver,pm.cv)
  result, all_model = LRmp.build_model_LR(train_x,train_y,test_x,test_y,pm.C,pm.penalty,pm.classweight,pm.multi_class,pm.solver,pm.cv)
elif pm.regression =='tree':
  file_name = 't_{}_tst{}_trn{}_dep_{}_splt_{}_crt_{}_cw_{}'.format(pm.time,pm.year,pm.interval,pm.max_dep,pm.split_type,pm.crt_type,pm.classweight)
  result, all_model = Tmp.build_model_tree(train_x,train_y,test_x,test_y,pm.max_dep,pm.split_type,pm.crt_type,pm.classweight)

1：No Injury
 2:Injury
 3:Fatal

p:  l1, C=1.0  
Logistic regression parameter {'C': 1.0, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'multinomial', 'n_jobs': None, 'penalty': 'l1', 'random_state': None, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Logistic regression Train Accuracy :  0.6255599182639853 

Logistic regression Test Accuracy :  0.6275702863432756 

p:  l2, C=1.0  
Logistic regression parameter {'C': 1.0, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'multinomial', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Logistic regression Train Accuracy :  0.6255530315029962 

Logistic regression Test Accuracy :  0.6275702863432756 



In [10]:
from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(train_y, all_model[0].predict(train_x))
print('sev_T1_P1'+str(cm_test[0,0]/np.sum(cm_test[0]))+'sev_T2_P2:'+str(cm_test[1,1]/np.sum(cm_test[1])))

sev_T1_P10.6430484451780281sev_T2_P2:0.6101075539808315


In [6]:
orders = ['Accuracy']  + [col for col in result if col != 'Accuracy']
result = result[orders]
file_name = 't_{}_tst{}_trn{}_Csize_{}_pnt_{}_cw_{}_mltc_{}_svr_{}_cv_{}'.format(pm.time,pm.year,pm.interval,len(pm.C),pm.penalty,pm.classweight,pm.multi_class,pm.solver,pm.cv)
result.to_csv(pm.regression+'_accuracy_result'+file_name+'.csv') 
import local_Result_Analysis as RA
allmodel_Acc_error_mean_sqrt,allmodel_victim_error_mean_sqrt,allmodel_Acc_error_mean,allmodel_victim_error_mean = RA.compare_model(test_data, test_x, all_model,pm.money_path)
allmodel_Acc_error_mean_sqrt.to_csv(file_name+'allmodel_Acc_error_mean_sqrt.csv')
allmodel_victim_error_mean_sqrt.to_csv(file_name+'allmodel_victim_error_mean_sqrt.csv')
allmodel_Acc_error_mean.to_csv(file_name+'allmodel_Acc_error_mean.csv')
allmodel_victim_error_mean.to_csv(file_name+'allmodel_victim_error_mean.csv')

IndexError: index 2 is out of bounds for axis 1 with size 2